# Image deconvolution using `RedLionFish/Deconwolf`

**To run image deconvolution using this notebook, you need a CUDA-compatible GPU with functional NVIDIA drivers, cuDNN, etc...**

To do image deconvolution, you will first need to know some parameters about your microscope. Take some time to carefully read the guide below and to collect some information before you attempt image deconvolution. This will save you a lot of computing time and frustration: using the wrong parameters will result in imaging artifacts, so be careful!

These parameters will be used to create a synthetic point-spread function (PSF). If you don't know what a PSF is and why it is important, please read here: ​​https://en.wikipedia.org/wiki/Point_spread_function

You will need to know:

`na` = numerical aperture of the used len

`m` = lens magnification

`ni0` = refraction index of the immersion medium

`res_lateral` = x,y resolution of the images 

`res_axial`: z-resolution of the stack. This is either the spacing between zplanes or the actual z resolution of your lens, whichever is larger. Remember that the actual Z resolution of your lens will match your experiment’s resolution only if you acquired the stacks at Nyquist conditions. (https://imb.uq.edu.au/research/facilities/microscopy/training-manuals/microscopy-online-resources/image-capture/nyquist-conditions)


## Introduce the PSF_metadata

Once you've collected the above information, you have to input it in the format indicated below. You can substitute the numbers with the actual values for your microscope.

**THIS STEP IS NEEDED IRRESPECTIVELY OF WHICHEVER OF THE FOLLOWING FUNCTIONS YOU NEED TO USE**

In [ ]:
#THIS IS FOR 5 COLOURS LEICA 20X
PSF_metadata = {'na':1.1,
'm':20,
'ni0':1.333,
'res_lateral':0.419,
'res_axial':0.859,
 'channels':{
 '0':{
    'wavelength':.809},
  '1':{
    'wavelength':.681},
  '2':{
    'wavelength':.555},
  '3':{
    'wavelength':.475},
  '4':{
    'wavelength':.390},
  '5':{
    'wavelength':.436}
     
 }
}

# Image deconvolution of Leica-exported tiffs

## `deconvolve_leica`
This function organises the raw images from a Leica experiment according to Cycle, Region, Tile, Channel, arranges them in individual 3D stacks, deconvolves each stack and save its maximum projection (default).  

The function mirrors the workflow of the `leica_mipping` function from the `ISS_preprocessing` module in the sense that organise the files in a similar way but performs a deconvolution before the maximum projection. The output files and folder have the same name and structure than conventionally projected files. This steps effectively substitutes the `leica_mipping` in preprocessing workflows where image deconvolution is needed.


The function takes the following arguments:

`input_dirs`: This will be a list of complete paths to the folder containing your imaging cycle. The folders need to be specified in the right order (ie. the first element of the list will be the folder where the first cycle of imaging is saved, and so on) The elements need to be separated by commas. The format for this variable is a `lst` of `str`.

`output_dir_prefix`:  This will be the path where you want the preprocessing output to be saved. Ideally, this should be associated with some type of unique project identifier. The format of this variable is `str`. In case multiple regions are being processed, the funcion will behave differently depending whether a trailing slash `/` is included or not in `output_dir_prefix`: 
- if a trailing slash is added, then subfolders for each one of the scanned regions will be created as `_R1`, `_R2`, etc...
- if a trailing slash is omitted, then `output_location` will be updated at each region iteration, and each one of the scanned regions will end up in a different `output_location` named as `output_location_R1`,`output_location_R2`, etc...

`cycles`: This will be a list of ISS cycles the input_dirs refer to. Each cycle is a `int` number, where 1 refers to cycle 1 and so on. If `cycle=None` the function will not work.

`deconvolution_method`: choose between `deconvolution_method='deconwolf'` (cpu based) and `deconvolution_method='redlionfish'` (gpu based). The Deconwolf executable is a command-line tool and needs to be installed before using, see deconwolf_install.txt

`image_dimensions`: this is a list of 2 elements [x,y], specifying the x and y sizes of each image (default=[2048, 2048])

`PSF_metadata`: refer to the example above. Metadata for the construction of a synthetic PSF

`mip`: Specifies if the deconvolved images need to be maximum projected (default = True). If `False` the deconvolved stacks are saved, however we do our ISS analysis in 2D so there's almost never a good reason to save the stack.

`mode`: `mode='autosaved'` for autosaved data in Leica microscope, `mode='exported'` for exported data in Leica microscope.

`deconvolve_leica` is able to handle multiple regions in the input files, and project them accordingly.

**Keep in mind that `leica_deconvolution` can only deconvolve ONE cycle at the time. This means that you need to manually specify the cycle number in the function for appropriate naming of the output files. There is also an option to embed this function in a loop to process multiple cycles if you want, but make sure to pass the right arguments.**

In [ ]:
# User specifies input_dirs, output_dir_prefix and the cycles corresponding to the input directories
# When processing one cycle only, the input_dirs is a list of one directory: ['/path/to/cycle1'] 
# and cycles is a list of one cycle: [1]

from ISS_deconvolution import deconvolution as dv

input_dirs = ['/path/to/cycle1/',
              '/path/to/cycle2/',
              '/path/to/cycle3/']
output_dir_prefix = '/path/to/my/output/folder/'

cycles = [1,2,3]

for cycle, input_dir in zip(cycles, input_dirs):

    dv.deconvolve_leica(input_dir, 
                        output_dir_prefix,
                        cycle=cycle,
                        deconvolution_method='deconwolf',
                        PSF_metadata=PSF_metadata, 
                        mip=True, 
                        mode='exported')

The output of `deconvolve_leica` are essentially maximum-projected images that have been previously deconvolved, so the output is seamlessly compatible with the downstream steps of normal preprocessing for Leica images: `leica_OME_tiff`, `ashlar_wrapper` and `tile_stitched_images`, so you can pick up the rest of the workflow in the `ISS_preprocessing` environment and notebooks


# Image deconvolution of CZI files from Zeiss

## `deconvolve_czi`
This function reads the CZI images from the Zeiss format, gets the images, arranges them in individual 3D stacks, deconvolves each stack and save its maximum projection (default).  

The function mirrors the workflow of the `process_czi` function from the `ISS_preprocessing` module in the sense that organise the files in a similar way but performs a deconvolution before the maximum projection. The output files and folder have the same name and structure than conventionally projected files. This steps effectively substitutes the `process_czi` in preprocessing workflows where image deconvolution is needed.


The function takes the following arguments:

`input_file`: the path to the CZI file that you want to preprocess, down to the czi file (included)

`outpath`: the folder where you want to save the maximum-projected images. Ideally this would be a `/mainoutputfolder/region/preprocessing/mipped/` folder structure, for consistency with our way of organising the data.

`cycle`: here you have to manually specify to which ISS cycle the images refer to. This is a `int` number, where 1 refers to cycle 1 and so on. If `cycle=0` the function will not work.

`tile_size_x` and `tile_size_y`: these refer to the size in pixel of your camera field of view. Most cameras are 2048x2048, so that's the default if you don't specify them, but adjust them if your camera has a different field of view size.

`chunk_size`: Depending on the GPU you have, you might be or not able to load the entire image stack onto the GPU RAM. If `chunk_size` is unspecified, the function will try to load the entire stack. If your kernel crashes before performing any actual work, the most likely cause is that you are running out of memory.
The solution to this RAM shortage is to chunk the stack in sub-stacks, deconvolve each independently and merge them afterwards. A typical `chunk_size` value that works for most GPU is [512,512] in XY. Z is automatically extracted from the data. If your GPU is very small you can try with [256,256] or even [128,128].

`mip`: Specifies if the deconvolved images need to be maximum projected (default = True). If `False` the deconvolved stacks are saved, however we do our ISS analysis in 2D so there's almost never a good reason to save the stack.

**Keep in mind that `deconvolve_czi` can only accept 1 file at the time, unlike `deconvolve_leica`. This means that you need to deconvolve ONE cycle at the time, and manually specify the cycle number in the function for appropriate naming of the output files. You can also embed this function in a loop to process multiple cycles if you want, but make sure to pass the right arguments.**

In [ ]:
from ISS_deconvolution import deconvolution as dv

dv.deconvolve_czi(input_file, 
                  outpath, 
                  image_dimensions=[2048, 2048], 
                  PSF_metadata=PSF_metadata, 
                  chunk_size=None,  
                  mip=True, 
                  cycle=0, 
                  tile_size_x=2048, 
                  tile_size_y=2048)

# Image deconvolution of .lif files from Leica

## `lif_deconvolution`

This function reads the .lif images from the Leica LIF format, gets the images, arranges them in individual 3D stacks, deconvolves each stack and save its maximum projection (default).  

The function mirrors the workflow of the `lif_mipping` function from the `ISS_preprocessing` module in the sense that organise the files in a similar way but performs a deconvolution before the maximum projection. The output files and folder have the same name and structure than conventionally projected files. This steps effectively substitutes the `lif_mipping` in preprocessing workflows where image deconvolution is needed.


The function takes the following arguments:

`input_dirs`: This will be a list of complete paths to the folder containing your lif file, NOT including the lif file. The folders need to be specified in the right order (ie. the first element of the list will be the folder where the first cycle of imaging is saved, and so on) The elements need to be separated by commas. The format for this variable is a `lst` of `str`.

`output_dir_prefix`:  This will be the path where you want the preprocessing output to be saved. Ideally, this should be associated with some type of unique project identifier. The format of this variable is `str`. In case multiple regions are being processed, the funcion will behave differently depending whether a trailing slash `/` is included or not in `output_dir_prefix`: 
- if a trailing slash is added, then subfolders for each one of the scanned regions will be created as `_R1`, `_R2`, etc...
- if a trailing slash is omitted, then `output_location` will be updated at each region iteration, and each one of the scanned regions will end up in a different `output_location` named as `output_location_R1`,`output_location_R2`, etc...

`cycles`: This will be a list of ISS cycles the input_dirs refer to. Each cycle is a `int` number, where 1 refers to cycle 1 and so on. If `cycle=None` the function will not work.

`PSF_metadata`: refer to the example above. Metadata for the construction of a synthetic PSF

`tile_size_x` and `tile_size_y`: these refer to the size in pixel of your camera field of view. Most cameras are 2048x2048, so that's the default if you don't specify them, but adjust them if your camera has a different field of view size.

`deconvolution_method`: choose between `deconvolution_method='deconwolf'` (cpu based) and `deconvolution_method='redlionfish'` (gpu based). The Deconwolf executable is a command-line tool and needs to be installed before using, see deconwolf_install.txt

`mip`: Specifies if the deconvolved images need to be maximum projected (default = True). If `False` the deconvolved stacks are saved, however we do our ISS analysis in 2D so there's almost never a good reason to save the stack.

**Keep in mind that `lif_deconvolution` can only deconvolve ONE cycle at the time. This means that you need to manually specify the cycle number in the function for appropriate naming of the output files. There is also an option to embed this function in a loop to process multiple cycles if you want, but make sure to pass the right arguments.**

In [ ]:
# User specifies input_dirs, output_dir_prefix and the cycles corresponding to the input directories
# When processing one cycle only, the input_dirs is a list of one directory: ['/path/to/cycle1'] 
# and cycles is a list of one cycle: [1]
from ISS_deconvolution import deconvolution as dv

input_dirs = ['/path/to/cycle1/',
              '/path/to/cycle2/',
              '/path/to/cycle3/']

cycles = [1,2,3]

output_dir_prefix = '/path/to/my/output/folder/'

for cycle, input_dir in zip(cycles, input_dirs):

    dv.deconvolve_lif(input_dir, 
                      output_dir_prefix, 
                      cycle=cycle,
                      PSF_metadata=PSF_metadata,  
                      tile_size_x=2048, 
                      tile_size_y=2048,
                      deconvolution_method='redlionfish',
                      mip=True)